# Part 3: Logistic Regression

In this part, we will be working with the wine dataset. This dataset contains 11 chemical features of various wines, along with experts' rating of that wine's quality. The quality scale technically runs from 1-10, but only 3-9 are actually used in the data.

**Reference**
P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. Modeling wine preferences by data mining from physicochemical properties.
In Decision Support Systems, Elsevier, 47(4):547-553, 2009. 

**Your goal is to fit a model able to classify a wine as good or bad quality.**

## The DataSet
The dataset contains the following features: 
1. fixed acidity
2. volatile acidity
3. citric acid
4. residual sugar
5. chlorides
6. free sulfur dioxide
7. total sulfur dioxide
8. density
9. pH
10. sulphates
11. alcohol

The 12th column of the data file corresponds to the output variable, which represents the quality of the wine (score between 0 and 10)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
wine_df = pd.read_csv('data/winequality-red.csv') #Reads a CSV file

## Exercise 1: Exploring the data

### Task 1.1: Using pandas for data exploration
As you did in linear regression, use the methods `head(params)` and `describe(params)` to explore the dataset.

In [3]:
#Your code here
wine_df.head(10)

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   
5            7.4              0.66         0.00             1.8      0.075   
6            7.9              0.60         0.06             1.6      0.069   
7            7.3              0.65         0.00             1.2      0.065   
8            7.8              0.58         0.02             2.0      0.073   
9            7.5              0.50         0.36             6.1      0.071   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   
5                 13.0                  40.0   0.9978  3.51       0.56   
6                 15.0                  59.0   0.9964  3.30       0.46   
7                 15.0                  21.0   0.9946  3.39       0.47   
8                  9.0                  18.0   0.9968  3.36       0.57   
9                 17.0                 102.0   0.9978  3.35       0.80   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5  
3      9.8        6  
4      9.4        5  
5      9.4        5  
6      9.4        5  
7     10.0        7  
8      9.5        7  
9     10.5        5

In [4]:
#Your code here
wine_df.describe()

fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.200000          0.640000     0.420000        2.600000   
max        15.900000          1.580000     1.000000       15.500000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  1599.000000          1599.000000           1599.000000  1599.000000   
mean      0.087467            15.874922             46.467792     0.996747   
std       0.047065            10.460157             32.895324     0.001887   
min       0.012000             1.000000              6.000000     0.990070   
25%       0.070000             7.000000             22.000000     0.995600   
50%       0.079000            14.000000             38.000000     0.996750   
75%       0.090000            21.000000             62.000000     0.997835   
max       0.611000            72.000000            289.000000     1.003690   

                pH    sulphates      alcohol      quality  
count  1599.000000  1599.000000  1599.000000  1599.000000  
mean      3.311113     0.658149    10.422983     5.636023  
std       0.154386     0.169507     1.065668     0.807569  
min       2.740000     0.330000     8.400000     3.000000  
25%       3.210000     0.550000     9.500000     5.000000  
50%       3.310000     0.620000    10.200000     6.000000  
75%       3.400000     0.730000    11.100000     6.000000  
max       4.010000     2.000000    14.900000     8.000000

### Task 1.2: Adding a good/bad label
The current database does not have a field indicating if a wine is good or bad. We will use the label quality to categorize wines into good or bad according to the following rule: 
    - A wine is considered good if its quality is equal or above 6.5
    
Create a new field in the DataFrame, denoted good, that reflects this rule.  

In [5]:
wine_df['good'] = wine_df['quality'].map(lambda x: True if x >= 6.5 else False) #Your code here

wine_df.good = wine_df.good.replace({True: 1, False: 0})
wine_df.head()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  good  
0      9.4        5     0  
1      9.8        5     0  
2      9.8        5     0  
3      9.8        6     0  
4      9.4        5     0

## Exercise 2: Data splits and further exploration
We will start first by splitting the data in two. For this exercise, we will only use a training and test set. We will omit the use of the validation set as we will train a single model and then check how well it generalizes.

In [6]:
wines_train, wines_test = train_test_split(wine_df, test_size=0.2, random_state=8, stratify=wine_df['good'])

X_train = wines_train.drop(['quality','good'], axis=1)
y_train = wines_train['good']

X_test = wines_test.drop(['quality','good'], axis=1)
y_test = wines_test['good']

X_train.head(15)

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
1411            6.4              0.47         0.40             2.4      0.071   
209            11.0              0.30         0.58             2.1      0.054   
632             7.6              0.41         0.14             3.0      0.087   
1182           10.2              0.40         0.40             2.5      0.068   
1310            7.5              0.61         0.26             1.9      0.073   
358            11.9              0.43         0.66             3.1      0.109   
159             6.8              0.60         0.18             1.9      0.079   
596            12.4              0.40         0.51             2.0      0.059   
207             7.8              0.57         0.31             1.8      0.069   
81              7.8              0.43         0.70             1.9      0.464   
826             7.5              0.27         0.34             2.3      0.050   
669            11.3              0.34         0.45             2.0      0.082   
1266            7.2              0.57         0.05             2.3      0.081   
1146            7.8              0.50         0.12             1.8      0.178   
939             6.2              0.46         0.17             1.6      0.073   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
1411                  8.0                  19.0  0.99630  3.56       0.73   
209                   7.0                  19.0  0.99800  3.31       0.88   
632                  21.0                  43.0  0.99640  3.32       0.57   
1182                 41.0                  54.0  0.99754  3.38       0.86   
1310                 24.0                  88.0  0.99612  3.30       0.53   
358                  10.0                  23.0  1.00000  3.15       0.85   
159                  18.0                  86.0  0.99680  3.59       0.57   
596                   6.0                  24.0  0.99940  3.04       0.60   
207                  26.0                 120.0  0.99625  3.29       0.53   
81                   22.0                  67.0  0.99740  3.13       1.28   
826                   4.0                   8.0  0.99510  3.40       0.64   
669                   6.0                  15.0  0.99880  2.94       0.66   
1266                 16.0                  36.0  0.99564  3.38       0.60   
1146                  6.0                  21.0  0.99600  3.28       0.87   
939                   7.0                  11.0  0.99425  3.61       0.54   

      alcohol  
1411     10.6  
209      10.5  
632      10.5  
1182     10.5  
1310      9.8  
358      10.4  
159       9.3  
596       9.3  
207       9.3  
81        9.4  
826      11.0  
669       9.2  
1266     10.3  
1146      9.8  
939      11.4

### Question 2.1: train_test_split
Explain what is the role of each of the parameters used in the following line of code:

`train_test_split(wine_df, test_size=0.2, random_state=8, stratify=wine_df['good'])`

Your answer here: 

As we saw before this method will split our dataset in two for training and testing. The first parameter, `wine_df`, is the array/dataset we want to split. 

The second parameter, `test_size=0.2`, means that we want that the test array is a 20% of the dataset and the training array is a 80% of the dataset. 

The third parameter, `random_state=8`, is provided to control the random number generator used, in this case, the random number generator is seeded by 8. 

The final parameter, `stratify=wine_df['good']`, is provided to ensure that both divisions have a proportional amount of good and bad wines.

The function scatter_matrix from pandas allows to visually explore the data. This can be useful to identify potential correlations across the input features (which are undesirable). 

In [7]:
from pandas.plotting import scatter_matrix
wines_train = wines_train.drop(['quality'], axis=1)
scatter_matrix(wines_train, figsize=(30,20))

array([[<Axes: xlabel='fixed acidity', ylabel='fixed acidity'>,
        <Axes: xlabel='volatile acidity', ylabel='fixed acidity'>,
        <Axes: xlabel='citric acid', ylabel='fixed acidity'>,
        <Axes: xlabel='residual sugar', ylabel='fixed acidity'>,
        <Axes: xlabel='chlorides', ylabel='fixed acidity'>,
        <Axes: xlabel='free sulfur dioxide', ylabel='fixed acidity'>,
        <Axes: xlabel='total sulfur dioxide', ylabel='fixed acidity'>,
        <Axes: xlabel='density', ylabel='fixed acidity'>,
        <Axes: xlabel='pH', ylabel='fixed acidity'>,
        <Axes: xlabel='sulphates', ylabel='fixed acidity'>,
        <Axes: xlabel='alcohol', ylabel='fixed acidity'>,
        <Axes: xlabel='good', ylabel='fixed acidity'>],
       [<Axes: xlabel='fixed acidity', ylabel='volatile acidity'>,
        <Axes: xlabel='volatile acidity', ylabel='volatile acidity'>,
        <Axes: xlabel='citric acid', ylabel='volatile acidity'>,
        <Axes: xlabel='residual sugar', ylabel='volati

### Question 2.2: Scatter matrix Analysis
Based on the observed plots, do you consider there are any correlations among features? 

Your answer here:

I would say there is a positive correlation between **fixed acidity** and **density** and a negative correlation between **fixed acidity** and **pH**. Also, I would say there is some correlation between **fixed acidity** and **citric acid** and **volatile acidity** and **citric acid**, bur it is lower than the mentioned before.

## Exercise 3: Training and Testing
We will  now proceed to train our logistic regression model.

In [8]:
from sklearn.linear_model import LogisticRegression
logistic_model = LogisticRegression(C=1000000, solver='newton-cg', max_iter=250).fit(X_train,y_train)
print(f'Logistic regression model coefficients:{logistic_model.coef_}\n')

Logistic regression model coefficients:[[ 2.34331451e-01 -2.96927696e+00  2.23198693e-02  2.70171328e-01
  -1.26499489e+01  1.47211314e-02 -1.99743826e-02 -1.98305588e+02
   3.37777261e-01  3.79651275e+00  7.33945532e-01]]



### Question set 3.1: Analysis of the coefficients
Based on the obtained coefficients:
1. Which features seem to have an important contribution towards considering a wine as good?
2. as bad?

Your answer here:

1. Taking into account the coefficients, the features that seems to have an important contribution towards considering a wine as good are: **sulphates**, **alcohol**, **pH** and **residual sugars** in that order.
2. To consider a wine as bad, the features that seems to have an important contribution are: **density**, **chlorides** and **citric acid**.

In [9]:
print(f'Accuracy:{logistic_model.score(X_test,y_test)}\n')

Accuracy:0.89375



### Question 3.2: Accuracy and generalizability
What accuracy did you obtain? Do you consider it good or bad? 

Your answer here:

Accuracy:0.89375 - I would say this is a high accuracy because it will have a much higher number of correct guesses than incorrect guesses.

### Task 3.1 Comparison against a dummy model
Suppose now that you build a dummy model that classifies all wines as bad. 

In [38]:
y_pred = np.zeros(len(y_test))

Estimate this model's accuracy:

In [39]:
#Your code here
sum = 0
y_test2= np.array(y_test)
for i in range(len(y_test2)):
    if y_pred[i] == y_test2[i]:
        sum +=1
accuracy = sum/len(y_test2)
#Your code ends here

print(f'Dummy model accuracy:{accuracy}\n')

Dummy model accuracy:0.865625



Given your results, has your original opinion about the trained model changed?

Your answer here:

The dummy model accuracy is 0.865625. It is approximately 0.025 lower than the logistic model accuracy. This difference is considerably small. Taking this into account, my original opinion has changed and for this case we need to improve the model to obtain a higher accuracy.